# Aula04 - Prática - Transformações Geométricas 2D no OpenGL

### Primeiro, vamos importar as bibliotecas necessárias.

In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm

ModuleNotFoundError: No module named 'glfw'

### Inicializando janela

In [ ]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### Shaders

In [ ]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

In [ ]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [ ]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [ ]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [ ]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [ ]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [ ]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [ ]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [ ]:
# preparando espaço para 4 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(4, [("position", np.float32, 2)])

# preenchendo as coordenadas de cada vértice

vertices['position'] = [
                            (+0.05, -0.05),
                            (+0.05, +0.05),
                            (-0.05, -0.05),
                            (-0.05, +0.05)
                        ]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot (buffer).

In [ ]:
# Request a buffer slot from GPU
buffer_VBO = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [ ]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)

### Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [ ]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [ ]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [ ]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Vamos pegar a localização da variável color para que possamos definir a cor em nosso laço da janela!

In [ ]:
loc_color = glGetUniformLocation(program, "color")
R = 0.7
G = 0.0
B = 0.2

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [ ]:
#### O código dessa célula não está sendo usado.

def key_event(window,key,scancode,action,mods):
    global t_x, t_y
    
    if key == 265: t_y += 0.01 #cima
    if key == 264: t_y -= 0.01 #baixo
    if key == 263: t_x -= 0.01 #esquerda
    if key == 262: t_x += 0.01 #direita


    
glfw.set_key_callback(window,key_event)


### Nesse momento, nós exibimos a janela!


In [ ]:
glfw.show_window(window)

### Loop principal da janela.

In [ ]:
import math

angulo = 0
t_x = 0
t_y = 0


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

glEnable(GL_BLEND);
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA);


while not glfw.window_should_close(window):
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)

    angulo -= 0.01
    c = math.cos(angulo)
    s = math.sin(angulo)

    mat_rotation = np.array([       c,   -s,  0.0, 0.0, 
                                    s,    c,  0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    #mat_final = multiplica_matriz(mat_rotation, mat_translation)


    # pede a localizacao da variavel mat_transformation no shader
    loc = glGetUniformLocation(program, "mat_transformation")
    
    # set mat_transformation com a mat_rotation
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_rotation)
    
    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) ## ative esse comando para enxergar os triângulos
    
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)
    
    
    glUniform4f(loc_color, R, G, B, 1.0) ### modificando a cor do objeto!
    
    glfw.swap_buffers(window)
    glfw.poll_events() 

glfw.terminate()

# Atividade Prática do dia:

Altere este notebook para que o resultado seja uma "nave 2D" laranja, ou seja, um <b>triângulo laranja</b> que se movimenta na tela de acordo com os eventos do teclado descritos a seguir.

<b>Setas:</b> movimentam a nave na tela (direita, esquerda, pra cima, pra baixo).

<b>Teclas 'a' e 's':</b> rotacionam a nave para a esquerda e para a direita.

<b>Teclas 'z' e 'x':</b> aumenta/diminui o tamanho da nave.

<b>Tecla 'r':</b> faz "reset", isto é, volta tudo como se nenhuma tecla tivesse sido pressionada após a renderização.

<b>Códigos das teclas: </b>

Direita: 262

Esquerda: 263

Cima: 265

Baixo: 264

a: 65

r: 82

s: 83

z: 90

x: 88

(Códigos de outras teclas: https://www.glfw.org/docs/3.3/group__keys.html)